In [1]:
%matplotlib inline

In [2]:
from eam_core.util import h5load
import os

[root                ][INFO   ]  Configured logging from /Users/jedpreist/CodingStuff/DIMPACT/eam-core-provenance/src/eam_core/logconf.yml (log_configuration.py:30)


In [3]:
d,m = h5load('../../output/country_test/20210719-163053/default/result_data_energy.hdf5')
# d,m = h5load('../../../output/1/CI_model/20200508-114814/result_data_energy.hdf5')
# os.getcwd()

[numexpr.utils       ][INFO   ]  NumExpr defaulting to 8 threads. (utils.py:157)


In [4]:
d.columns

MultiIndex(levels=[['CDN', 'Internet Network', 'Origin Datacentre'], ['gigawatt_hour']],
           codes=[[2, 0, 1], [0, 0, 0]],
           names=[None, 'unit'])

In [5]:
lci = d['CDN'].groupby(level=['time']).quantile(.05)

In [6]:
uci=d['CDN'].groupby(level=['time']).quantile(.95)

KeyError: 'Cellular Infrastructure'

In [ ]:
_mean = d['Cellular Infrastructure'].mean(level='time')

In [ ]:
d['Cellular Infrastructure'].index.unique(level='time')

In [ ]:
lci['gigawatt_hour']

In [ ]:
ax = _mean.plot()
ax.fill_between(d['Cellular Infrastructure'].index.unique(level='time'), lci['gigawatt_hour'], uci['gigawatt_hour'], alpha=.25)

In [ ]:
d.pint.quantify(level=-1)

In [ ]:
d['number_iplayer_4_5g_hh']

In [ ]:
import json
from ngmodel.SimulationRunner import SimulationRunner
from ngmodel.YamlLoader import YamlLoader
from ngmodel.common_graphical_analysis import load_metadata
from ngmodel.core import complex_decoder, complex_encoder, ServiceModel, ScalarDataSource, ExcelDataSource, \
    SimulationControl

import numpy as np
from ngmodel.util import create_output_folder, load_as_df_qantity

In [ ]:
model_output_directory = 'output/example/20190517-160221/default'

In [ ]:
base_dir = '.'
metadata = load_metadata(model_output_directory, base_dir=base_dir)
load_data = load_as_df_qantity
variable = 'energy'
q_data = load_data(f'{model_output_directory}/result_data_{variable}.hdf5')
data = q_data.m
unit = str(q_data.units)

# monthly mean values for all processes
mean = data.mean(level='time').mean()


In [ ]:
_d = data.mean(level='time').sum()

In [ ]:
{k:v for k,v in _d.items()}

In [ ]:
list(_d.keys())

In [ ]:
list(_d.values)